In [93]:
# email2inventory
# read evergreen email
# this script will fail if scripts/email2inventory/data/evergreen-20200930.txt doesn't exist

import os
import re

def evergreen_read(filename):
    date = ""
    product_flag = False
    position = 0
    items = {}
    f = open(filename)
    lines = f.readlines()
    while not product_flag:
        line = lines.pop(0).strip()
        m = re.match(r"Date: (\w* \d+, \d{4})", line)
        if m:
            date = m.group(1)
        if line == "Product":
            product_flag = True
    line = lines.pop(0).strip()
    assert line == "Quantity"
    line = lines.pop(0).strip()
    assert line == "Price"
    while product_flag:
        m = re.match(r"(.*) - 1 Week", lines.pop(0).strip())
        try:
            itemname = m.group(1)
        except:
            break
        quantity = lines.pop(0).strip()
        price = lines.pop(0).strip()
        items[itemname] = [quantity,price]
    f.close()
    return({'items': items, 'date' : date})
    
print(evergreen_read(os.path.join("email_data/evergreen-20200930.txt")))

{'items': {"Red's Best Hake": ['2', '$32.00']}, 'date': 'September 30, 2020'}


In [166]:
# email2inventory
# read kistler email
# this script will fail if scripts/email2inventory/data/kistler-20200912.txt doesn't exist

import os
import re
import requests
import datetime

client_secret = subprocess.check_output("./sortly_key.sh").strip()
post_url = 'https://api.sortly.co/api/v1/items'

def kistler_read(filename):
    date = ""
    product_flag = False
    position = 0
    items = {}
    f = open(filename)
    lines = f.readlines()
    while not product_flag and lines:
        line = lines.pop(0).strip()
# grabbing the date from the email header
        m = re.match(r"Date: (\w* \d+, \d{4})", line)
        if m:
            date = datetime.datetime.strptime(m.group(1), '%B %d, %Y')
            iso_date = str(date_time_obj).replace('+00:00', 'Z')
# this version still assumes that there is a single item which is a several sets of wines
# in a bundle - need to generalize
        elif line == "Total":
            product_flag = True
        while product_flag:
            m = re.search(r"(\d+)-Bottle Mixed Package\*$", lines.pop(0).strip())
            items = get_package(int(m.group(1)),lines)
            product_flag = False
    f.close()
    return({'items' : items, 'iso_date' : iso_date})

def get_package(bottles,lines):
    inventory = {}
    while bottles > 0:
        line = lines.pop(0).strip()
        m = re.match(r"\•(\d+) btls (\d{4}) (.*)$",line)
        inventory[m.group(3)] = [int(m.group(1)), m.group(2)]
        bottles -= int(m.group(1))
    return inventory
    
inventory = kistler_read(os.path.join("email_data/kistler-20200912.txt"))

for item in inventory['items']:
#    print(item)
#    print(inventory['items'][item])
#    print(inventory['items'][item][0])
# posting to Sortly API
    post_url = 'https://api.sortly.co/api/v1/items'
    data = {'name' : item, 'quantity' : inventory['items'][item][0], 'type' : 'item', 'tags' : [{'name' : 'test'}, {'name' : 'wine'}], 'custom_attribute_values': [{'value': inventory['iso_date'], 'custom_attribute_id': 170931, 'custom_attribute_name': 'Purchase Date'}]}
    response = requests.post(post_url, json=data, headers=headers)

McCrea Vineyard Chardonnay Sonoma Mountain
[2, '2018']
2
Durell Vineyard Chardonnay Sonoma Coast
[2, '2018']
2
Laguna Ridge Vineyard Pinot Noir Russian River Valley
[2, '2018']
2


This uses OAUTH2 to get the two folders from the Sortly API and then open an item and get the image.

Boris figured out how to do it using `urllib.request`, but I think I should be able to do it using `requests`.  Also, I think we're doing some sort of brute force OAUTH and that there is something more complicated but proper - maybe using requests. Something to try to figure out.

I moved the client key to sortly_key.sh as a file that prints the key. I ignored it in GitHub and am not syncing it.

Now trying to generalize for the farmers to you email

In [89]:
# email2inventory
# read farmers to you email
# this script will fail if scripts/email2inventory/data/f2u-20201006.txt doesn't exist

import os
import re
import requests
import datetime
import subprocess
import urllib.request
import json
import io
import subprocess

client_secret = subprocess.check_output("./sortly_key.sh").strip()
working_path = os.getenv("HOME") + '/GitHub/joi.github.io/__scripts/'
post_url = 'https://api.sortly.co/api/v1/items'
product_pattern = 'Item'
pop_fields = ['Item', 'Wt', '$/lb', 'Qty', 'Cost']
sortly_fields = ['name', 'notes', '', 'quantity', 'price']
end_items = ""
data_file = 'email_data/f2u-20201006.txt'


def get_file(filename):
    date = ""
    product_flag = False
    position = 0
    items = {}
    
    print("opening file" + data_file)
    f = open(working_path + filename)
    lines = f.readlines()
    
    while not product_flag and position < len(lines):
        line = lines[position].strip()
        position += 1
# grabbing the date from the email header
        m = re.match(r"Date: (\w* \d+, \d{4})", line)
        if m:
            date_time_obj = datetime.datetime.strptime(m.group(1), '%B %d, %Y')
            iso_date = str(date_time_obj).replace('00:00:00', 'Z')
# this version still assumes that there is a single item which is a several sets of wines
# in a bundle - need to generalize
        elif line == product_pattern:
            product_flag = True
# iterate past each line with a field from the header of the item list
            for field in pop_fields[1:]:
                line = lines[position].strip()
                position += 1
        while product_flag:
# iterate through fields. Will fail if first field isn't name
            for field_sortly in sortly_fields:
                if field_sortly == '':
                    position +=1
                elif field_sortly == 'name':
                    name = lines[position].strip()
                    position += 1
                    if name not in items:
                        items[name] = {}
                    else:
                        i = 1
                        while name + " " + str(i) in items:
                            i += 1
                        name = name + " " + str(i)
                        items[name] = {}
                else:
                    if field_sortly == 'quantity':
                        items[name][field_sortly] = int(lines[position].strip())
                    elif field_sortly == 'price':
                        items[name][field_sortly] = float(lines[position].strip()[1:])
                    else:
                        items[name][field_sortly] = lines[position].strip()
                    position += 1
            line = lines[position].strip()
            if line == end_items:
                product_flag = False
    f.close()
    return items, iso_date

def get_package(bottles,lines):
    inventory = {}
    while bottles > 0:
        line = lines.pop(0).strip()
        m = re.match(r"\•(\d+) btls (\d{4}) (.*)$",line)
        inventory[m.group(3)] = [int(m.group(1)), m.group(2)]
        bottles -= int(m.group(1))
    return inventory

def post_shortly(inventory):
    url = 'https://api.sortly.co/api/v1/items'
    client_secret = subprocess.check_output("./sortly_key.sh").strip()
    headers = {'Accept': 'application/json','Authorization': client_secret}
    data = {}
    for item in inventory:
        data = {'name' : str(item)}
        for key, value in inventory[item].items():
            data[key] = value
        data['type'] = 'item'
        data['tags'] = [{'name' : 'test'}]
        print(data)
        response = requests.post(url, json=data, headers=headers)

inventory, date = get_file(data_file)
post_shortly(inventory)


opening fileemail_data/f2u-20201006.txt
{'name': 'Chicken - 4-5.5 lbs Medium', 'notes': '5.49 lbs', 'quantity': 1, 'price': 25.25, 'type': 'item', 'tags': [{'name': 'test'}]}
{'name': 'Milk - Creamline Organic - Strfrd', 'notes': '-', 'quantity': 1, 'price': 6.4, 'type': 'item', 'tags': [{'name': 'test'}]}
{'name': 'Buttermilk - Organic', 'notes': '-', 'quantity': 1, 'price': 3.9, 'type': 'item', 'tags': [{'name': 'test'}]}
{'name': 'Cream - Heavy Organic', 'notes': '-', 'quantity': 1, 'price': 7.9, 'type': 'item', 'tags': [{'name': 'test'}]}
{'name': 'Cheese - Melting Blend', 'notes': '-', 'quantity': 1, 'price': 11.9, 'type': 'item', 'tags': [{'name': 'test'}]}
{'name': 'Cheese - Triple Cream Organic', 'notes': '-', 'quantity': 1, 'price': 6.4, 'type': 'item', 'tags': [{'name': 'test'}]}
{'name': 'Cheese - Bayley Hazen Blue', 'notes': '-', 'quantity': 1, 'price': 12.9, 'type': 'item', 'tags': [{'name': 'test'}]}
{'name': 'Cheese - Goat Feta', 'notes': '-', 'quantity': 1, 'price': 11.

In [75]:
# trying to access Sortly API
import urllib.request
import json
import requests
import io
import subprocess
from PIL import Image

client_secret = subprocess.check_output("./sortly_key.sh").strip()

headers = {
  'Accept': 'application/json',
  'Authorization': client_secret
}
response = requests.get('https://api.sortly.co/api/v1/items?per_page=&page=&folder_id=&include=', headers=headers).json()
print("Folder: " + response['data'][0]['name'])
next_url = response['meta']['next_page_url']
response = requests.get(next_url, headers=headers).json()
print(response)
print("Folder: " + response['data'][0]['name'])
next_url = response['meta']['next_page_url']
response = requests.get('https://api.sortly.co/api/v1/items/17481596?include=custom_attributes%2Cphotos', headers=headers).json()
photo_url = response['data']['photos'][0]['url']
print(response['data']['custom_attribute_values'])
response = requests.get(photo_url)
img = Image.open(io.BytesIO(response.content))
print("here's an item with a photo")
img.show()
# Posting test object
url = 'https://api.sortly.co/api/v1/items'

data = {'name' : "test object", 'type' : 'item', 'tags' : [{'name' : 'test'}]}
# print(url, data, headers)
response = requests.post(url, json=data, headers=headers)

Folder: Food
{'meta': {'page': 2, 'next_page_url': 'https://api.sortly.co/api/v1/items?folder_id=&include=&page=3&per_page=', 'total_pages': 8, 'total_count': 8}, 'data': [{'id': 17356162, 'name': 'Durable Things', 'price': None, 'quantity': None, 'min_quantity': None, 'notes': None, 'parent_id': None, 'sid': 'S3YNFR0002', 'label_url': None, 'label_url_type': None, 'label_url_extra': None, 'label_url_extra_type': None, 'tag_names': [], 'type': 'folder', 'created_at': '2020-09-30T13:32:45.181Z', 'updated_at': '2020-10-04T11:42:09.498Z'}]}
Folder: Durable Things
[{'value': '2020-09-05T04:00:00.000Z', 'custom_attribute_id': 170931, 'custom_attribute_name': 'Purchase Date'}, {'value': None, 'custom_attribute_id': 171873, 'custom_attribute_name': 'Product URL'}]
here's an item with a photo
